# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


# Get GCS PATH

In [3]:
GCS_PATH="gs://kds-e2d3c8fe9f8348f76af9288c67efea6938fff49c6d2271152b97bbf1" # @param
print(GCS_PATH)


gs://kds-e2d3c8fe9f8348f76af9288c67efea6938fff49c6d2271152b97bbf1


# Fixed Parameters

In [4]:
TFIDEN      =   'OcrDTF'# @param
DATA_DIM    =   128     # @param
IMG_DIM     =   64      # @param
NB_CHANNEL  =   3       # @param
BATCH_SIZE  =   128     # @param
BUFFER_SIZE =   512     # @param
TRAIN_DATA  =   9600    # @param
EVAL_DATA   =   2400    # @param
TEST_DATA   =   3000    # @param
NB_CLASSES  =   50      # @param

GCS_PATH='{}/{}'.format(GCS_PATH,TFIDEN)
print(GCS_PATH)

gs://kds-e2d3c8fe9f8348f76af9288c67efea6938fff49c6d2271152b97bbf1/OcrDTF


# Data Input Function

In [5]:
import tensorflow as tf 
import os 


print(tf.__version__)

def data_input_fn(mode): 
    '''
      This Function generates data from gcs
    '''
    
    def _parser(example):
        feature ={  'image'  : tf.io.FixedLenFeature((),tf.string) ,
                    'label'  : tf.io.FixedLenFeature((),tf.int64)
        }    
        parsed_example=tf.io.parse_single_example(example,feature)
        image_raw=parsed_example['image']
        image=tf.image.decode_png(image_raw,channels=NB_CHANNEL)
        image=tf.cast(image,tf.float32)/255.0
        image=tf.reshape(image,(DATA_DIM,DATA_DIM,NB_CHANNEL))
        image=tf.image.resize(image,[IMG_DIM,IMG_DIM])
        
        idx = tf.cast(parsed_example['label'], tf.int32)
        label=tf.one_hot(idx,NB_CLASSES,dtype=tf.float32)
        return image,label

    gcs_pattern=os.path.join(GCS_PATH,mode,'*.tfrecord')
    file_paths = tf.io.gfile.glob(gcs_pattern)
    dataset = tf.data.TFRecordDataset(file_paths)
    dataset = dataset.map(_parser)
    dataset = dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

2.3.0


# Demo Data availability Check

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train_ds  =   data_input_fn("Train")
eval_ds   =   data_input_fn("Eval")
test_ds   =   data_input_fn("Test")


for x,y in train_ds.take(1):
    data=np.squeeze(x[0])
    plt.imshow(data)
    plt.show()
    print('Image Batch Shape:',x.shape)
    print('Target Batch Shape:',y.shape)
    print(y[0])
    

" \nfor x,y in train_ds.take(1):\n    data=np.squeeze(x[0])\n    plt.imshow(data)\n    plt.show()\n    print('Image Batch Shape:',x.shape)\n    print('Target Batch Shape:',y.shape)\n    print(y[0])\n"

# TPU Check

In [7]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.109.12.10:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.109.12.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.109.12.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# MODEL
* Select one model from the link [tensorflow official](https://www.tensorflow.org/api_docs/python/tf/keras/applications/)

In [8]:
model_name='DenseNet121' # @param

with strategy.scope():
    base = tf.keras.applications.DenseNet121(include_top=False,
                                          weights='imagenet', 
                                          pooling='avg',
                                          input_shape=(IMG_DIM,IMG_DIM,NB_CHANNEL))
    inp = base.input
    x   = base.output
    x   = tf.keras.layers.Dense(NB_CLASSES, 
                                activation='softmax',
                                name='predictions')(x)
    
    model = tf.keras.Model(inp,x)
    model.compile(optimizer="Adam",
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])



model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________________

# Training

In [9]:
EPOCHS=250 # @param
TOTAL_DATA=TRAIN_DATA+EVAL_DATA
STEPS_PER_EPOCH = TOTAL_DATA//BATCH_SIZE
EVAL_STEPS      = EVAL_DATA//BATCH_SIZE

In [10]:
WEIGHT_PATH=os.path.join(os.getcwd(),f'{model_name}.h5') 

# reduces learning rate on plateau
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,
                                                  cooldown= 10,
                                                  patience=10,
                                                  verbose =1,
                                                  min_lr=0.1e-5)

mode_autosave = tf.keras.callbacks.ModelCheckpoint(WEIGHT_PATH,
                                                  monitor='val_accuracy', 
                                                  mode = 'max', 
                                                  save_best_only=True, 
                                                  verbose=1, 
                                                  period =1)

# stop learining as metric on validatopn stop increasing
early_stopping = tf.keras.callbacks.EarlyStopping(patience=15, 
                                                  verbose=1, 
                                                  mode = 'auto') 

callbacks = [mode_autosave, lr_reducer,early_stopping ]



In [11]:
history = model.fit(train_ds,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=eval_ds,
                    validation_steps=EVAL_STEPS,
                    callbacks=callbacks)

InvalidArgumentError: ignored

# Saving and Scoring

In [ ]:

model.save_weights(WEIGHT_PATH)


def plot_history(history):
    """
    Plots model training history 
    """
    fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
    ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
    ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax_loss.legend()
    ax_acc.plot(history.epoch, history.history["accuracy"], label="Train IoU")
    ax_acc.plot(history.epoch, history.history["val_accuracy"], label="Validation IoU")
    ax_acc.legend()
    plt.show()

    # show history
plot_history(history)